In [1]:
!pip install -q scikit-multiflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.6/450.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from skmultiflow.drift_detection import ADWIN, DDM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from matplotlib.colors import ListedColormap
from multiprocessing.pool import ThreadPool
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
from contextlib import suppress
from collections import Counter
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from random import shuffle
import seaborn as sns
from time import time
import pandas as pd
import numpy as np
import warnings
import scipy.io
import pickle
import sys
import gc
import os
import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data'
code_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/Codes/Shared Codes'
results_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/Results'
feature_selection_results = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/feature_selection_results'
sys.path.insert(0,code_path)
from oselm import set_use_know
from SharedFunctions import main,generate_oselm_models

In [5]:
filenames = ['kddcup99_csv.csv','ISCX2012.csv','CSE-CIC2018.csv','CICIDS2017.csv','7recurrentDrift.csv', 'blip.csv', 'incrementalDrift.csv',
             '7gradualDrift.csv', '7suddenDrift.csv']
filenames = list(map(lambda x: os.path.join(data_path, x), filenames))

In [6]:
data_name = ['kddcup99','ISCX2012','CSE-CIC2018','CICIDS2017','7recurrentDrift', 'blip', 'incrementalDrift','7gradualDrift', '7suddenDrift']

In [7]:
# # use_know = False
# # set_use_know(use_know)
# # path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
# # os.makedirs(path, exist_ok=True)
# # for f_name,d_name in zip(filenames,data_name):
# #   DFS_results_path = os.path.join(feature_selection_results,d_name)
# #   main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
# #        feature_selection=["feature_evolving",'DFS_feature_selection'], result_save_path=path,DFS_results_path=DFS_results_path)

In [8]:
DATA_NUMBER=0
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'],result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : kddcup99_csv.csv ======================


0it [00:00, ?it/s]

Evolving DFS
1 {'accuracy': 0.9961513790891597, 'precision': 1.0, 'recall': 0.9927095990279465, 'f1-score': 0.9963414634146343, 'auc': 0.9963547995139732}


  0%|          | 0/7832 [00:00<?, ?it/s]

...

Evolving DFS
8 {'accuracy': 0.9930379746835443, 'precision': 1.0, 'recall': 0.986335403726708, 'f1-score': 0.9931207004377736, 'auc': 0.993167701863354}


  0%|          | 0/7892 [00:00<?, ?it/s]

Evolving DFS
9 {'accuracy': 0.9949335022165928, 'precision': 1.0, 'recall': 0.9901356350184957, 'f1-score': 0.9950433705080546, 'auc': 0.9950678175092478}


  0%|          | 0/7790 [00:00<?, ?it/s]

Evolving DFS
10 {'accuracy': 0.9961489088575096, 'precision': 0.9987012987012988, 'recall': 0.9935400516795866, 'f1-score': 0.9961139896373058, 'auc': 0.9961322707377525}


In [9]:
DATA_NUMBER=1
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'],result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : ISCX2012.csv ======================


0it [00:00, ?it/s]

Evolving DFS
1 {'accuracy': 0.8744561839651958, 'precision': 0.9861830742659758, 'recall': 0.74640522875817, 'f1-score': 0.8497023809523809, 'auc': 0.868463277886194}


  0%|          | 0/8018 [00:00<?, ?it/s]

...

Evolving DFS
8 {'accuracy': 0.869538077403246, 'precision': 0.9916107382550335, 'recall': 0.7433962264150943, 'f1-score': 0.8497483824586628, 'auc': 0.8686002197750813}


  0%|          | 0/8138 [00:00<?, ?it/s]

Evolving DFS
9 {'accuracy': 0.8556511056511057, 'precision': 0.9949324324324325, 'recall': 0.7174177831912302, 'f1-score': 0.8336871903750884, 'auc': 0.8568501555361355}


  0%|          | 0/8122 [00:00<?, ?it/s]

Evolving DFS
10 {'accuracy': 0.8596923076923076, 'precision': 0.9933993399339934, 'recall': 0.7288135593220338, 'f1-score': 0.840782122905028, 'auc': 0.861903650749878}


In [ ]:
DATA_NUMBER=2
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : CSE-CIC2018.csv ======================


0it [00:00, ?it/s]

Evolving DFS
1 {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'auc': 0.5}


  0%|          | 0/8000 [00:00<?, ?it/s]

...

  0%|          | 0/15970 [00:00<?, ?it/s]

Evolving DFS
4 {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'auc': 1.0}


  0%|          | 0/15964 [00:00<?, ?it/s]

Evolving DFS
5 {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'auc': 1.0}


  0%|          | 0/15982 [00:00<?, ?it/s]

In [ ]:
DATA_NUMBER=3
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

In [ ]:
DATA_NUMBER=4
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

In [ ]:
DATA_NUMBER=5
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

In [ ]:
DATA_NUMBER=6
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

In [ ]:
DATA_NUMBER=7
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

In [ ]:
DATA_NUMBER=8
ChunkNumber=0
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)